In [ ]:
import torch
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
0.0
0.0
0.0


In [ ]:
# !pip install transformers

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import torch
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaModel
import math
import Model_Import_6
from torch import optim
import random
from sklearn.model_selection import train_test_split
from transformers import AdamW
import numpy as np
# from transformers import WarmupLinearSchedule
import gc


In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
6.43825664
6.421954048
0.016302592


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
def load_stuff(context_tensors, generator_logits, text_ids_loc):
    context = torch.load(context_tensors, map_location=lambda storage, loc: storage.cuda(0))
    logits = torch.load(generator_logits, map_location=lambda storage, loc: storage.cuda(0))
    text_ids = torch.load(text_ids_loc, map_location=lambda storage, loc: storage.cuda(0))
    return context, logits, text_ids

In [ ]:
# def load_stuff(context_tensors, generator_logits, text_ids_loc):
#     context = torch.load(context_tensors)
#     logits = torch.load(generator_logits)
#     text_ids = torch.load(text_ids_loc)
#     return context, logits, text_ids

In [ ]:
R_neg_embeds, neg_logits, neg_token_ids = load_stuff('R_neg_embeds.pt', 'neg_logits.pt', 'neg_token_ids.pt')
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
29.69567232
29.573845504
0.121826816


In [ ]:
# R_pos_embeds, pos_logits, pos_token_ids = load_stuff('R_pos_embeds.pt', 'pos_logits.pt', 'pos_token_ids.pt')
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved
# print(t/1000000000)
# print(r/1000000000)
# print(a/1000000000)
# print(f/1000000000)

In [ ]:
# R_embeds_train, R_embeds_test, logits_train, logits_test, token_ids_train, token_ids_test = train_test_split(R_neg_embeds+R_pos_embeds, neg_logits+pos_logits, neg_token_ids+pos_token_ids,  test_size=0.2, random_state=42)

In [ ]:
R_neg_embeds_train, R_neg_embeds_test, neg_logits_train, neg_logits_test, neg_token_ids_train, neg_token_ids_test = train_test_split(R_neg_embeds, neg_logits, neg_token_ids,  test_size=0.2, random_state=42)

In [ ]:
print(len(R_neg_embeds_train))
print(len(neg_logits_train))
print(len(neg_token_ids_train))

10000
10000
10000


In [ ]:
# print(len(R_embeds_train))
# print(len(logits_train))
# print(len(token_ids_train))

In [ ]:
def train_one_style(model, optimizer, context_embeds_list, logits_list, token_ids_list, epochs, num_samples = 100):
    CELoss = nn.CrossEntropyLoss()
    total_count = 0
    for epoch in range(epochs):
        
        to_shuffle = list(zip(logits_list, token_ids_list))

        random.shuffle(to_shuffle)

        logits_list, token_ids_list = zip(*to_shuffle)

        model.train()
        ag_loss_epoch = 0
        epoch_count = 0
        for example in range(len(logits_list)):
            random_context_samples = random.sample(context_embeds_list, num_samples) # could use another context to see what happens
            stacked_context_sample = torch.stack(random_context_samples, dim = 0)
            # print(stacked_context_sample.shape)
            optimizer.zero_grad()
            network_output = model(stacked_context_sample.to(device), logits_list[example].to(device))
            if token_ids_list[example]['input_ids'].shape[1] == 1:
                print("ONE text id")
                continue
            shifted_network_output = network_output[..., :-1, :].contiguous()
            shifted_text_ids = token_ids_list[example]['input_ids'][..., 1:].contiguous().to(device)
            loss = CELoss(shifted_network_output.view(-1, shifted_network_output.size(-1)), shifted_text_ids.view(-1))
            ag_loss_epoch += loss
            epoch_count += 1
            total_count += 1
            loss.backward()
            optimizer.step()
            if example%1000 == 0:
                print(".", end = "")
        print(f"Epoch: {epoch}, Epoch Examples: {epoch_count}")
        print(f"TRAIN LOSS: {ag_loss_epoch / len(logits_list)}")
        # print(f"DEV LOSS: {full_dev_loss}")
        print("----------------------------------------")

In [ ]:
def train_one_style_w_dev(model, optimizer, context_embeds_list, logits_list, token_ids_list, epochs, dev_logits, dev_context, dev_token_ids, num_samples = 100):
    CELoss = nn.CrossEntropyLoss()
    total_count = 0
    for epoch in range(epochs):
        
        to_shuffle = list(zip(logits_list, token_ids_list))

        random.shuffle(to_shuffle)

        logits_list, token_ids_list = zip(*to_shuffle)

        model.train()
        ag_loss_epoch = 0
        epoch_count = 0
        for example in range(len(logits_list)):
            random_context_samples = random.sample(context_embeds_list, num_samples) # could use another context to see what happens
            stacked_context_sample = torch.stack(random_context_samples, dim = 0)
            # print(stacked_context_sample.shape)
            optimizer.zero_grad()
            network_output = model(stacked_context_sample.to(device), logits_list[example].to(device))
            if token_ids_list[example]['input_ids'].shape[1] == 1:
                print("ONE text id")
                continue
            shifted_network_output = network_output[..., :-1, :].contiguous()
            shifted_text_ids = token_ids_list[example]['input_ids'][..., 1:].contiguous().to(device)
            loss = CELoss(shifted_network_output.view(-1, shifted_network_output.size(-1)), shifted_text_ids.view(-1))
            ag_loss_epoch += loss
            epoch_count += 1
            total_count += 1
            loss.backward()
            optimizer.step()
            if example%1000 == 0:
                print(".", end = "")
        model.eval()
        CELoss_dev = nn.CrossEntropyLoss()
        dev_loss_acum = 0
        for dev_example in range(len(dev_logits)):
            random_context_samples_dev = random.sample(dev_context, num_samples) # need to make sure dev samples are the same... !!!!
            stacked_context_sample_dev = torch.stack(random_context_samples_dev, dim = 0)
            dev_network_output = model(stacked_context_sample_dev, dev_logits[dev_example])
            shifted_network_output_dev = dev_network_output[..., :-1, :].contiguous()
            shifted_text_ids_dev = dev_token_ids[dev_example]['input_ids'][..., 1:].contiguous()
            dev_loss = CELoss_dev(shifted_network_output_dev.view(-1, shifted_network_output_dev.size(-1)), shifted_text_ids_dev.view(-1))
            dev_loss_acum += dev_loss.item()
        full_dev_loss = dev_loss_acum / len(dev_logits)
        
        print(f"Epoch: {epoch}, Epoch Examples: {epoch_count}")
        print(f"TRAIN LOSS: {ag_loss_epoch / len(logits_list)}")
        print(f"DEV LOSS: {full_dev_loss}")
        print("----------------------------------------")

In [ ]:
def one_style_generate(prompt, tokenizer, SAT_model, GPT_transformer, context_to_sample, num_samples = 100, num_tokens_to_generate = 1, sen_to_generate = 10):
  #Put models in eval mode
    SAT_model.eval()
    GPT_transformer.eval()
    
    for sent in range(sen_to_generate):
        random_context_samples = random.sample(context_to_sample, num_samples)
        model_context_input =  torch.stack(random_context_samples, dim = 0)
        expierment_list = []
        # tokenize prompt
        current_tokenization = tokenizer.encode(prompt, return_tensors = 'pt').to(device)

        for generation in range(num_tokens_to_generate):
            # put tokenized prompt through GPT_transformer to get GPT Logits
            GPT_logits = GPT_transformer(current_tokenization).last_hidden_state.squeeze()

            # put model_context_input and the GPT Logits into SAT model
            adjusted_output = SAT_model(model_context_input, GPT_logits)

            # Funtional softmax the SAT model output
            SM_adjusted_output = torch.nn.functional.softmax(adjusted_output, dim = 1)

            # argmax to get predicted token
            # predicted_tokens = torch.argmax(SM_adjusted_output, dim =1)

            # get topk tokens
            top_predicted_tokens = torch.topk(SM_adjusted_output[-1], 5, dim =0).indices
            top_predicted_tokens_prob = torch.topk(SM_adjusted_output[-1], 5, dim =0).values
            top_predicted_tokens_numpy = top_predicted_tokens.cpu().detach().numpy()
            top_predicted_tokens_prob_numpy = top_predicted_tokens_prob.cpu().detach().numpy()
            token_prob_sum = np.sum(top_predicted_tokens_prob_numpy)
            token_distribution = top_predicted_tokens_prob_numpy/token_prob_sum
            
            predicted_token =  torch.from_numpy(np.array(np.random.choice(top_predicted_tokens_numpy, p = token_distribution))).to(device)
            
            # print(top_predicted_tokens)
             # tok_k_predicted_words = tokenizer.decode(top_predicted_tokens, skip_special_tokens=True)
            decoded_tokens = []
            for token_k in top_predicted_tokens:
                decoded_tokens.append(tokenizer.decode(token_k, skip_special_tokens=True))
                
            top_predicted_tokens_prob_cpu = top_predicted_tokens_prob.cpu()
            del top_predicted_tokens_prob
            expierment_list.append((top_predicted_tokens_prob_cpu, decoded_tokens))


                
            #dif way of getting top predicted
            # predicted_token = top_predicted_tokens[0]
            # print(current_tokenization[-1][0])
            # print(predicted_token)
            # if predicted_token == 247 or predicted_token == current_tokenization[-1][0]:
            if predicted_token == 247:
                print("WEIRD TOKEN PREDICTED")
                predicted_token = top_predicted_tokens[1]

            # print(torch.amax(SM_adjusted_output[-1]))

            # print(predicted_tokens[-1].unsqueeze(0).unsqueeze(0))
            # print(current_tokenization)

            current_tokenization = torch.cat((current_tokenization, predicted_token.unsqueeze(0).unsqueeze(0)), 1)
            del predicted_token
            gc.collect()
            # print(current_tokenization)

        # decode
        for i, beam in enumerate(current_tokenization):
            # print(f"{i}: {tokenizer.decode(beam)}")
            # print(f"{i}: {current_tokenization}")
            print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)}")
        del current_tokenization
        del random_context_samples
        del model_context_input
        gc.collect()
        for prediction in expierment_list:
            print(prediction)
        print("\n\n")
    # for i, beam in enumerate(predicted_tokens):
    #     # if i == 0:
    #     #   continue
    #     print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)} token_id: {beam}")


In [ ]:
# def train_two_style_w_dev(model, optimizer, context_embeds_list, logits_list, token_ids_list, epochs, dev_logits, dev_context, dev_token_ids, num_samples = 100):
#     CELoss = nn.CrossEntropyLoss()
#     total_count = 0
#     for epoch in range(epochs):
        
#         to_shuffle = list(zip(logits_list, token_ids_list))

#         random.shuffle(to_shuffle)

#         logits_list, token_ids_list = zip(*to_shuffle)

#         model.train()
#         ag_loss_epoch = 0
#         epoch_count = 0
#         for example in range(len(logits_list)):
#             random_context_samples = random.sample(context_embeds_list, num_samples) # could use another context to see what happens
#             stacked_context_sample = torch.stack(random_context_samples, dim = 0)
#             # print(stacked_context_sample.shape)
#             optimizer.zero_grad()
#             network_output = model(stacked_context_sample.to(device), logits_list[example].to(device))
#             if token_ids_list[example]['input_ids'].shape[1] == 1:
#                 print("ONE text id")
#                 continue
#             shifted_network_output = network_output[..., :-1, :].contiguous()
#             shifted_text_ids = token_ids_list[example]['input_ids'][..., 1:].contiguous().to(device)
#             loss = CELoss(shifted_network_output.view(-1, shifted_network_output.size(-1)), shifted_text_ids.view(-1))
#             ag_loss_epoch += loss
#             epoch_count += 1
#             total_count += 1
#             loss.backward()
#             optimizer.step()
#             if example%1000 == 0:
#                 print(".", end = "")
#         model.eval()
#         CELoss_dev = nn.CrossEntropyLoss()
#         dev_loss_acum = 0
#         for dev_example in range(len(dev_logits)):
#             random_context_samples_dev = random.sample(dev_context, num_samples) # need to make sure dev samples are the same... !!!!
#             stacked_context_sample_dev = torch.stack(random_context_samples_dev, dim = 0)
#             dev_network_output = model(stacked_context_sample_dev, dev_logits[dev_example])
#             shifted_network_output_dev = dev_network_output[..., :-1, :].contiguous()
#             shifted_text_ids_dev = dev_token_ids[dev_example]['input_ids'][..., 1:].contiguous()
#             dev_loss = CELoss_dev(shifted_network_output_dev.view(-1, shifted_network_output_dev.size(-1)), shifted_text_ids_dev.view(-1))
#             dev_loss_acum += dev_loss.item()
#         full_dev_loss = dev_loss_acum / len(dev_logits)
        
#         print(f"Epoch: {epoch}, Epoch Examples: {epoch_count}")
#         print(f"TRAIN LOSS: {ag_loss_epoch / len(logits_list)}")
#         print(f"DEV LOSS: {full_dev_loss}")
#         print("----------------------------------------")

In [ ]:
# neg_alone_model = Model_Import_6.Test_skip_norm_model(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device)
# neg_optimizer = optim.Adam(neg_alone_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
# train_one_style(neg_alone_model, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
# neg_alone_model = Model_Import_6.Test_skip_norm_model(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device)
# neg_optimizer = optim.Adam(neg_alone_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
# train_one_style_w_dev(neg_alone_model, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 20, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
29.69567232
29.573845504
0.121826816


In [ ]:
prompt = "I thought"

In [ ]:
# num_heads = 16
num_heads = 8

In [ ]:
pytorch_stacked_two_cross= Model_Import_6.MultiHeadModel_PyTorch_Stacked(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_two_cross.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far lr=0.0001,  weight_decay=.0001

In [ ]:
train_one_style_w_dev(pytorch_stacked_two_cross, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 10, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.2931504249572754
DEV LOSS: 3.246247928380966
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.2244300842285156
DEV LOSS: 3.234160268497467
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.175823211669922
DEV LOSS: 3.237887247514725
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 3.122915267944336
DEV LOSS: 3.240265000295639
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 3.06491756439209
DEV LOSS: 3.254282879114151
----------------------------------------
..........Epoch: 5, Epoch Examples: 10000
TRAIN LOSS: 3.003410577774048
DEV LOSS: 3.2699905432224274
----------------------------------------
..........Epoch: 6, Epoch Examples: 10000
TRAIN LOSS: 2.937988519668579
DEV LOSS: 3.285699222135544
----------------------------------------
..........E

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_two_cross, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50, sen_to_generate = 100)

0: I thought that this film was pretty boring. The only reason I watched it was because I was curious about it. But the acting wasn't too bad. The story is not that great either. The acting was good enough for a B-Movie movie. I
(tensor([0.3836, 0.1720, 0.0901, 0.0772, 0.0473], grad_fn=<ToCopyBackward0>), [' this', ' that', ' I', ' it', ' the'])
(tensor([0.3320, 0.2363, 0.0582, 0.0561, 0.0228], grad_fn=<ToCopyBackward0>), [' this', ' the', ' I', ' it', ' a'])
(tensor([0.4145, 0.1970, 0.1690, 0.0495, 0.0165], grad_fn=<ToCopyBackward0>), [' movie', ' film', ' was', ' is', ' would'])
(tensor([0.6391, 0.0585, 0.0580, 0.0559, 0.0176], grad_fn=<ToCopyBackward0>), [' was', ' is', ' had', ' would', ' could'])
(tensor([0.1007, 0.0720, 0.0701, 0.0441, 0.0339], grad_fn=<ToCopyBackward0>), [' a', ' very', ' pretty', ' so', ' really'])
(tensor([0.1270, 0.0969, 0.0869, 0.0650, 0.0502], grad_fn=<ToCopyBackward0>), [' bad', ' awful', ' funny', ' boring', ' atro'])
(tensor([0.3230, 0.2953, 0.1496, 0.04

In [ ]:
pytorch_stacked_triple_alt= Model_Import_6.MultiHeadModel_PyTorch_Stacked_Triple_Alt(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_triple_alt.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far

In [ ]:
train_one_style_w_dev(pytorch_stacked_triple_alt, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

.

KeyboardInterrupt: 

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_triple_alt, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 100)

0: I thought it would have a lot to say, and it has. It is not just about what it says about me. I am a person, a person with feelings and a life, a human. It's about the fact I have a family and I love my wife, my kids and I have friends and family and friends of my family and I love my friends. I have friends that are not like this, friends who have a lot more in their life. It is not about what it is.
0: I thought it would have a lot to say, and it has. It is not just about what it says about me. I am a person, a person with feelings and a life, a human. It's about the fact I have a family and I love my wife, my kids and I have friends and family and friends of my family and I love my friends. I have friends that are not like this, friends who have a lot more in their life. It is not about what it is.
0: I thought it would have a lot to say, and it has. It is not just about what it says about me. I am a person, a person with feelings and a life, a human. It's about the fact I have a

In [ ]:
pytorch_stacked_cross_positional= Model_Import_6.MultiHeadModel_PyTorch_Stacked_Positional(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_cross_positional.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far

In [ ]:
train_one_style_w_dev(pytorch_stacked_cross_positional, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.3605806827545166
DEV LOSS: 3.2910168653964997
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.248650312423706
DEV LOSS: 3.280663930606842
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.177448034286499
DEV LOSS: 3.2733029612064364
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 3.1034562587738037
DEV LOSS: 3.288507435274124
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 3.025240421295166
DEV LOSS: 3.30116226682663
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_cross_positional, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 100)

0: I thought it was good to have some of those guys in there, but I was really disappointed to not see a whole lot from them in this series, I mean I was hoping to have some good things from those two guys, and I was not. It's not like I was really disappointed with the rest, I was really looking for some help. I didn´ve seen any of those other two, and it´´´´d be good for me if I got a few more guys. But it didn't
WEIRD TOKEN PREDICTED
0: I thought it was good to have some of those guys in there, but I was a big part in getting the movie to where we are now, so it's not really fair. But it's not a good movie, but I don´ve got a feeling it will get a good review when the movie is released in theaters."I was in a relationship at that point and we had been dating a year, so it wasn't a good time. But it wasn´ve a lot of good times and a good movie
0: I thought it was good to have some of those guys in there, but I was really disappointed to not see a whole lot from them in this series, I

In [ ]:
pytorch_stacked_cross= Model_Import_6.MultiHeadModel_PyTorch_Stacked(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_cross.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


In [ ]:
train_one_style_w_dev(pytorch_stacked_cross, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.2936744689941406
DEV LOSS: 3.245574431180954
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.223997116088867
DEV LOSS: 3.2370047632217407
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.1751065254211426
DEV LOSS: 3.239879521083832
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 3.121413230895996
DEV LOSS: 3.2442128241062163
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 3.0633127689361572
DEV LOSS: 3.2566996613502504
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_cross, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

0: I now know what it's all like. It was like watching paint drying on the canvas. It's not the worst movie ever made. But I can only imagine the real story. I was very young, I didn`ve just graduated highschool and was working
0: I now know what it's all like. It was like watching paint drying on the wall of the house of the devil, the devil's own home, and the movie is so awful, I couldn`ve never even been a fan. The movie was awful,
0: I now know what it's all like. It was like watching paint drying on the canvas. It's not the worst movie ever made. But I can only imagine the real story. I was very young, I didn`ve just graduated highschool and was working
0: I now know what it's all like. It was like watching paint drying on the canvas. It's not the worst movie ever made. But I can only imagine the real story. I was very young, I didn`ve just graduated highschool and was working
0: I now know what it's all like. It was like watching paint drying on the canvas. It's not the worst mo

In [ ]:
pytorch_stacked_one_not_cross= Model_Import_6.MultiHeadModel_PyTorch_Stacked_One_Alt(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_one_not_cross.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


In [ ]:
train_one_style_w_dev(pytorch_stacked_one_not_cross, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 1.9614466428756714
DEV LOSS: 0.82565540587008
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 0.582233190536499
DEV LOSS: 0.5536075608596206
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 0.36249151825904846
DEV LOSS: 0.45461988470181824
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 0.27979013323783875
DEV LOSS: 0.4196639986075461
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 0.2359471619129181
DEV LOSS: 0.4072770089544356
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_one_not_cross, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 10, sen_to_generate = 1 )

0: The movie movie movie movie movie movie movie movie movie movie movie
(tensor([9.9798e-01, 1.0970e-03, 1.6808e-04, 1.4440e-04, 8.4563e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', ' movies', 'movie', ' the', ' theater'])
(tensor([9.9904e-01, 7.2250e-04, 4.6235e-05, 3.1694e-05, 1.5173e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', 'movie', ' movies', ' picture', ' theater'])
(tensor([9.9865e-01, 9.7820e-04, 4.1188e-05, 3.6795e-05, 2.6668e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', 'movie', ' movies', ' game', ' picture'])
(tensor([9.9883e-01, 7.1711e-04, 5.0114e-05, 4.7247e-05, 2.7799e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', 'movie', ' game', ' movies', ' picture'])
(tensor([9.9917e-01, 3.6677e-04, 4.7966e-05, 4.2018e-05, 2.5491e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', 'movie', ' game', ' movies', ' picture'])
(tensor([9.9896e-01, 3.2737e-04, 6.8437e-05, 5.9356e-05, 3.5

In [ ]:
pytorch_basic_positional= Model_Import_6.MultiHeadModel_PyTorch_Positional(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_basic_positional.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


In [ ]:
train_one_style_w_dev(pytorch_basic_positional, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.3547589778900146
DEV LOSS: 3.295817288970947
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.257972478866577
DEV LOSS: 3.276579099750519
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.1971757411956787
DEV LOSS: 3.2754206407546995
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 3.1409406661987305
DEV LOSS: 3.2812213646888733
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 3.0841259956359863
DEV LOSS: 3.29154849858284
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic_positional, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

NameError: name 'pytorch_basic_positional' is not defined

In [ ]:
num_heads = 8
pytorch_basic= Model_Import_6.MultiHeadModel_PyTorch(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_basic.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


In [ ]:
prompt = "I thought"

In [ ]:
train_one_style_w_dev(pytorch_basic, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 3, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.2852327823638916
DEV LOSS: 3.245586234807968
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.2262442111968994
DEV LOSS: 3.2330075203418733
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.182225227355957
DEV LOSS: 3.2341347328186036
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100,  num_tokens_to_generate = 50, sen_to_generate = 10 )

0: I thought this film was a good movie. I liked the cast, I liked the script. It's not the most entertaining movie ever made, but if you're looking for something to get you through a tough time in life, this one's for the job
(tensor([0.1569, 0.1447, 0.1324, 0.1013, 0.0751], grad_fn=<ToCopyBackward0>), [' I', ' this', ' it', ' that', ' the'])
(tensor([0.4233, 0.1911, 0.1710, 0.0260, 0.0237], grad_fn=<ToCopyBackward0>), [' movie', ' was', ' film', ' would', ' is'])
(tensor([0.5825, 0.0998, 0.0550, 0.0250, 0.0232], grad_fn=<ToCopyBackward0>), [' was', ' would', ' had', ' could', ' is'])
(tensor([0.1002, 0.0690, 0.0670, 0.0490, 0.0438], grad_fn=<ToCopyBackward0>), [' a', ' so', ' terrible', ' awful', ' bad'])
(tensor([0.0743, 0.0586, 0.0508, 0.0455, 0.0369], grad_fn=<ToCopyBackward0>), [' good', ' great', ' little', ' waste', ' disappointment'])
(tensor([0.1203, 0.1025, 0.0779, 0.0592, 0.0581], grad_fn=<ToCopyBackward0>), [' idea', ' example', ' one', ' movie', ' film'])
(tensor([0.2707,

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 39.41 GiB total capacity; 31.10 GiB already allocated; 3.56 MiB free; 31.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
pytorch_basic= Model_Import_6.MultiHeadModel_PyTorch(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
optimizer = optim.RAdam(pytorch_basic.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


NameError: name 'num_heads' is not defined

In [ ]:
train_one_style_w_dev(pytorch_basic, optimizer, R_embeds_train, logits_train, token_ids_train, 5, logits_test, R_embeds_test,  token_ids_test) ## dev implemented

....................

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_mm)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, R_pos_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
MH_basic= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1]/num_heads)).to(device) #
# MH_basic= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8).to(device) #

neg_optimizer = optim.Adam(MH_basic.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style_w_dev(MH_basic, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 4, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

In [ ]:
####################### R_neg_embeds_train, R_neg_embeds_test, neg_logits_train, neg_logits_test, neg_token_ids_train, neg_token_ids_test
#(model, optimizer, context_embeds_list, logits_list, token_ids_list, epochs, dev_logits, dev_context, dev_token_ids, num_samples = 100)

In [ ]:
neg_alone_model = Model_Import_6.Test_skip_norm_model(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device)
neg_optimizer = optim.Adam(neg_alone_model.parameters(), lr=0.0001,  weight_decay=0.001)

In [ ]:
train_one_style(neg_alone_model, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
neg_alone_model_deeper = Model_Import_6.DeeperModel_skip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device)
neg_optimizer = optim.Adam(neg_alone_model_deeper.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(neg_alone_model_deeper, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
neg_alone_model_wide = Model_Import_6.WiderModel_skip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device) # 2 wide -> 1:46  4 wide ->3:00
neg_optimizer = optim.Adam(neg_alone_model_wide.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(neg_alone_model_wide, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
neg_alone_model_wide_deep = Model_Import_6.WiderDeeperModel_2(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device) #
neg_optimizer = optim.Adam(neg_alone_model_wide_deep.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(neg_alone_model_wide_deep, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
import torch
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

In [ ]:
WiderBlock_8_model= Model_Import_6.WiderBlock_8(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device) #
neg_optimizer = optim.Adam(WiderBlock_8_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(WiderBlock_8_model, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
one_head_8_wide_model= Model_Import_6.ProposedModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = neg_logits[0].shape[1]*8).to(device) #
neg_optimizer = optim.Adam(one_head_8_wide_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(one_head_8_wide_model, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
prompt = "The movie"

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, WiderBlock_8_model, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
prompt = "Overall I thought"

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, WiderBlock_8_model, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 100)

In [ ]:
prompt = "I sat"

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, one_head_8_wide_model, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
# WiderDeeperModel_Alt

In [ ]:
Wide_deep_alt= Model_Import_6.WiderDeeperModel_Alt(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = neg_logits[0].shape[1]).to(device) #
neg_optimizer = optim.Adam(Wide_deep_alt.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(Wide_deep_alt, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
narrow_M_head= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1]/4)).to(device) #
neg_optimizer = optim.Adam(narrow_M_head.parameters(), lr=0.00001,  weight_decay=0.001) # 2:00 per epoch

In [ ]:
train_one_style(narrow_M_head, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
 neg_logits[0].shape[1]/8

In [ ]:
R_neg_embeds[0].shape[0]/8

In [ ]:
Wide_S_head= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 1, attention_dim = int(neg_logits[0].shape[1]*8)).to(device) #
neg_optimizer = optim.Adam(Wide_S_head.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(Wide_S_head, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
wide_M_head= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(wide_M_head.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(wide_M_head, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
wider_8_fixedM= Model_Import_6.WiderBlock_8_FixedM(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device) #
neg_optimizer = optim.Adam(wider_8_fixedM.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(wider_8_fixedM, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, wider_8_fixedM, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, narrow_M_head, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
middleskip_8= Model_Import_6.MultiHeadModel_WOFinalBlockSkip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(middleskip_8.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(middleskip_8, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
twoskip_8= Model_Import_6.MultiHeadModel_TwoSkip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(twoskip_8.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(twoskip_8, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
noskip= Model_Import_6.MultiHeadModel_WOAnyBlockSkip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(noskip.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(noskip, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
extraskip= Model_Import_6.MultiHeadModel_ExtraSkip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(extraskip.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(extraskip, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
import torch
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
0.0
0.0
0.0


In [ ]:
# !pip install transformers

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import torch
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaModel
import math
import Model_Import_6
from torch import optim
import random
from sklearn.model_selection import train_test_split
from transformers import AdamW
import numpy as np
# from transformers import WarmupLinearSchedule
import gc


In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
6.43825664
6.421954048
0.016302592


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
def load_stuff(context_tensors, generator_logits, text_ids_loc):
    context = torch.load(context_tensors, map_location=lambda storage, loc: storage.cuda(0))
    logits = torch.load(generator_logits, map_location=lambda storage, loc: storage.cuda(0))
    text_ids = torch.load(text_ids_loc, map_location=lambda storage, loc: storage.cuda(0))
    return context, logits, text_ids

In [ ]:
# def load_stuff(context_tensors, generator_logits, text_ids_loc):
#     context = torch.load(context_tensors)
#     logits = torch.load(generator_logits)
#     text_ids = torch.load(text_ids_loc)
#     return context, logits, text_ids

In [ ]:
R_neg_embeds, neg_logits, neg_token_ids = load_stuff('R_neg_embeds.pt', 'neg_logits.pt', 'neg_token_ids.pt')
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
29.69567232
29.573845504
0.121826816


In [ ]:
# R_pos_embeds, pos_logits, pos_token_ids = load_stuff('R_pos_embeds.pt', 'pos_logits.pt', 'pos_token_ids.pt')
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved
# print(t/1000000000)
# print(r/1000000000)
# print(a/1000000000)
# print(f/1000000000)

In [ ]:
# R_embeds_train, R_embeds_test, logits_train, logits_test, token_ids_train, token_ids_test = train_test_split(R_neg_embeds+R_pos_embeds, neg_logits+pos_logits, neg_token_ids+pos_token_ids,  test_size=0.2, random_state=42)

In [ ]:
R_neg_embeds_train, R_neg_embeds_test, neg_logits_train, neg_logits_test, neg_token_ids_train, neg_token_ids_test = train_test_split(R_neg_embeds, neg_logits, neg_token_ids,  test_size=0.2, random_state=42)

In [ ]:
print(len(R_neg_embeds_train))
print(len(neg_logits_train))
print(len(neg_token_ids_train))

10000
10000
10000


In [ ]:
# print(len(R_embeds_train))
# print(len(logits_train))
# print(len(token_ids_train))

In [ ]:
def train_one_style(model, optimizer, context_embeds_list, logits_list, token_ids_list, epochs, num_samples = 100):
    CELoss = nn.CrossEntropyLoss()
    total_count = 0
    for epoch in range(epochs):
        
        to_shuffle = list(zip(logits_list, token_ids_list))

        random.shuffle(to_shuffle)

        logits_list, token_ids_list = zip(*to_shuffle)

        model.train()
        ag_loss_epoch = 0
        epoch_count = 0
        for example in range(len(logits_list)):
            random_context_samples = random.sample(context_embeds_list, num_samples) # could use another context to see what happens
            stacked_context_sample = torch.stack(random_context_samples, dim = 0)
            # print(stacked_context_sample.shape)
            optimizer.zero_grad()
            network_output = model(stacked_context_sample.to(device), logits_list[example].to(device))
            if token_ids_list[example]['input_ids'].shape[1] == 1:
                print("ONE text id")
                continue
            shifted_network_output = network_output[..., :-1, :].contiguous()
            shifted_text_ids = token_ids_list[example]['input_ids'][..., 1:].contiguous().to(device)
            loss = CELoss(shifted_network_output.view(-1, shifted_network_output.size(-1)), shifted_text_ids.view(-1))
            ag_loss_epoch += loss
            epoch_count += 1
            total_count += 1
            loss.backward()
            optimizer.step()
            if example%1000 == 0:
                print(".", end = "")
        print(f"Epoch: {epoch}, Epoch Examples: {epoch_count}")
        print(f"TRAIN LOSS: {ag_loss_epoch / len(logits_list)}")
        # print(f"DEV LOSS: {full_dev_loss}")
        print("----------------------------------------")

In [ ]:
def train_one_style_w_dev(model, optimizer, context_embeds_list, logits_list, token_ids_list, epochs, dev_logits, dev_context, dev_token_ids, num_samples = 100):
    CELoss = nn.CrossEntropyLoss()
    total_count = 0
    for epoch in range(epochs):
        
        to_shuffle = list(zip(logits_list, token_ids_list))

        random.shuffle(to_shuffle)

        logits_list, token_ids_list = zip(*to_shuffle)

        model.train()
        ag_loss_epoch = 0
        epoch_count = 0
        for example in range(len(logits_list)):
            random_context_samples = random.sample(context_embeds_list, num_samples) # could use another context to see what happens
            stacked_context_sample = torch.stack(random_context_samples, dim = 0)
            # print(stacked_context_sample.shape)
            optimizer.zero_grad()
            network_output = model(stacked_context_sample.to(device), logits_list[example].to(device))
            if token_ids_list[example]['input_ids'].shape[1] == 1:
                print("ONE text id")
                continue
            shifted_network_output = network_output[..., :-1, :].contiguous()
            shifted_text_ids = token_ids_list[example]['input_ids'][..., 1:].contiguous().to(device)
            loss = CELoss(shifted_network_output.view(-1, shifted_network_output.size(-1)), shifted_text_ids.view(-1))
            ag_loss_epoch += loss
            epoch_count += 1
            total_count += 1
            loss.backward()
            optimizer.step()
            if example%1000 == 0:
                print(".", end = "")
        model.eval()
        CELoss_dev = nn.CrossEntropyLoss()
        dev_loss_acum = 0
        for dev_example in range(len(dev_logits)):
            random_context_samples_dev = random.sample(dev_context, num_samples) # need to make sure dev samples are the same... !!!!
            stacked_context_sample_dev = torch.stack(random_context_samples_dev, dim = 0)
            dev_network_output = model(stacked_context_sample_dev, dev_logits[dev_example])
            shifted_network_output_dev = dev_network_output[..., :-1, :].contiguous()
            shifted_text_ids_dev = dev_token_ids[dev_example]['input_ids'][..., 1:].contiguous()
            dev_loss = CELoss_dev(shifted_network_output_dev.view(-1, shifted_network_output_dev.size(-1)), shifted_text_ids_dev.view(-1))
            dev_loss_acum += dev_loss.item()
        full_dev_loss = dev_loss_acum / len(dev_logits)
        
        print(f"Epoch: {epoch}, Epoch Examples: {epoch_count}")
        print(f"TRAIN LOSS: {ag_loss_epoch / len(logits_list)}")
        print(f"DEV LOSS: {full_dev_loss}")
        print("----------------------------------------")

In [ ]:
def one_style_generate(prompt, tokenizer, SAT_model, GPT_transformer, context_to_sample, num_samples = 100, num_tokens_to_generate = 1, sen_to_generate = 10):
  #Put models in eval mode
    SAT_model.eval()
    GPT_transformer.eval()
    
    for sent in range(sen_to_generate):
        random_context_samples = random.sample(context_to_sample, num_samples)
        model_context_input =  torch.stack(random_context_samples, dim = 0)
        expierment_list = []
        # tokenize prompt
        current_tokenization = tokenizer.encode(prompt, return_tensors = 'pt').to(device)

        for generation in range(num_tokens_to_generate):
            # put tokenized prompt through GPT_transformer to get GPT Logits
            GPT_logits = GPT_transformer(current_tokenization).last_hidden_state.squeeze()

            # put model_context_input and the GPT Logits into SAT model
            adjusted_output = SAT_model(model_context_input, GPT_logits)

            # Funtional softmax the SAT model output
            SM_adjusted_output = torch.nn.functional.softmax(adjusted_output, dim = 1)

            # argmax to get predicted token
            # predicted_tokens = torch.argmax(SM_adjusted_output, dim =1)

            # get topk tokens
            top_predicted_tokens = torch.topk(SM_adjusted_output[-1], 5, dim =0).indices
            top_predicted_tokens_prob = torch.topk(SM_adjusted_output[-1], 5, dim =0).values
            top_predicted_tokens_numpy = top_predicted_tokens.cpu().detach().numpy()
            top_predicted_tokens_prob_numpy = top_predicted_tokens_prob.cpu().detach().numpy()
            token_prob_sum = np.sum(top_predicted_tokens_prob_numpy)
            token_distribution = top_predicted_tokens_prob_numpy/token_prob_sum
            
            predicted_token =  torch.from_numpy(np.array(np.random.choice(top_predicted_tokens_numpy, p = token_distribution))).to(device)
            
            # print(top_predicted_tokens)
             # tok_k_predicted_words = tokenizer.decode(top_predicted_tokens, skip_special_tokens=True)
            decoded_tokens = []
            for token_k in top_predicted_tokens:
                decoded_tokens.append(tokenizer.decode(token_k, skip_special_tokens=True))
                
            top_predicted_tokens_prob_cpu = top_predicted_tokens_prob.cpu()
            del top_predicted_tokens_prob
            expierment_list.append((top_predicted_tokens_prob_cpu, decoded_tokens))


                
            #dif way of getting top predicted
            # predicted_token = top_predicted_tokens[0]
            # print(current_tokenization[-1][0])
            # print(predicted_token)
            # if predicted_token == 247 or predicted_token == current_tokenization[-1][0]:
            if predicted_token == 247:
                print("WEIRD TOKEN PREDICTED")
                predicted_token = top_predicted_tokens[1]

            # print(torch.amax(SM_adjusted_output[-1]))

            # print(predicted_tokens[-1].unsqueeze(0).unsqueeze(0))
            # print(current_tokenization)

            current_tokenization = torch.cat((current_tokenization, predicted_token.unsqueeze(0).unsqueeze(0)), 1)
            del predicted_token
            gc.collect()
            # print(current_tokenization)

        # decode
        for i, beam in enumerate(current_tokenization):
            # print(f"{i}: {tokenizer.decode(beam)}")
            # print(f"{i}: {current_tokenization}")
            print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)}")
        del current_tokenization
        del random_context_samples
        del model_context_input
        gc.collect()
        for prediction in expierment_list:
            print(prediction)
        print("\n\n")
    # for i, beam in enumerate(predicted_tokens):
    #     # if i == 0:
    #     #   continue
    #     print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)} token_id: {beam}")


In [ ]:
# def train_two_style_w_dev(model, optimizer, context_embeds_list, logits_list, token_ids_list, epochs, dev_logits, dev_context, dev_token_ids, num_samples = 100):
#     CELoss = nn.CrossEntropyLoss()
#     total_count = 0
#     for epoch in range(epochs):
        
#         to_shuffle = list(zip(logits_list, token_ids_list))

#         random.shuffle(to_shuffle)

#         logits_list, token_ids_list = zip(*to_shuffle)

#         model.train()
#         ag_loss_epoch = 0
#         epoch_count = 0
#         for example in range(len(logits_list)):
#             random_context_samples = random.sample(context_embeds_list, num_samples) # could use another context to see what happens
#             stacked_context_sample = torch.stack(random_context_samples, dim = 0)
#             # print(stacked_context_sample.shape)
#             optimizer.zero_grad()
#             network_output = model(stacked_context_sample.to(device), logits_list[example].to(device))
#             if token_ids_list[example]['input_ids'].shape[1] == 1:
#                 print("ONE text id")
#                 continue
#             shifted_network_output = network_output[..., :-1, :].contiguous()
#             shifted_text_ids = token_ids_list[example]['input_ids'][..., 1:].contiguous().to(device)
#             loss = CELoss(shifted_network_output.view(-1, shifted_network_output.size(-1)), shifted_text_ids.view(-1))
#             ag_loss_epoch += loss
#             epoch_count += 1
#             total_count += 1
#             loss.backward()
#             optimizer.step()
#             if example%1000 == 0:
#                 print(".", end = "")
#         model.eval()
#         CELoss_dev = nn.CrossEntropyLoss()
#         dev_loss_acum = 0
#         for dev_example in range(len(dev_logits)):
#             random_context_samples_dev = random.sample(dev_context, num_samples) # need to make sure dev samples are the same... !!!!
#             stacked_context_sample_dev = torch.stack(random_context_samples_dev, dim = 0)
#             dev_network_output = model(stacked_context_sample_dev, dev_logits[dev_example])
#             shifted_network_output_dev = dev_network_output[..., :-1, :].contiguous()
#             shifted_text_ids_dev = dev_token_ids[dev_example]['input_ids'][..., 1:].contiguous()
#             dev_loss = CELoss_dev(shifted_network_output_dev.view(-1, shifted_network_output_dev.size(-1)), shifted_text_ids_dev.view(-1))
#             dev_loss_acum += dev_loss.item()
#         full_dev_loss = dev_loss_acum / len(dev_logits)
        
#         print(f"Epoch: {epoch}, Epoch Examples: {epoch_count}")
#         print(f"TRAIN LOSS: {ag_loss_epoch / len(logits_list)}")
#         print(f"DEV LOSS: {full_dev_loss}")
#         print("----------------------------------------")

In [ ]:
# neg_alone_model = Model_Import_6.Test_skip_norm_model(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device)
# neg_optimizer = optim.Adam(neg_alone_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
# train_one_style(neg_alone_model, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
# neg_alone_model = Model_Import_6.Test_skip_norm_model(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device)
# neg_optimizer = optim.Adam(neg_alone_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
# train_one_style_w_dev(neg_alone_model, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 20, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
29.69567232
29.573845504
0.121826816


In [ ]:
prompt = "I thought"

In [ ]:
# num_heads = 16
num_heads = 8

In [ ]:
pytorch_stacked_two_cross= Model_Import_6.MultiHeadModel_PyTorch_Stacked(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_two_cross.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far lr=0.0001,  weight_decay=.0001

In [ ]:
train_one_style_w_dev(pytorch_stacked_two_cross, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 10, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.2931504249572754
DEV LOSS: 3.246247928380966
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.2244300842285156
DEV LOSS: 3.234160268497467
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.175823211669922
DEV LOSS: 3.237887247514725
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 3.122915267944336
DEV LOSS: 3.240265000295639
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 3.06491756439209
DEV LOSS: 3.254282879114151
----------------------------------------
..........Epoch: 5, Epoch Examples: 10000
TRAIN LOSS: 3.003410577774048
DEV LOSS: 3.2699905432224274
----------------------------------------
..........Epoch: 6, Epoch Examples: 10000
TRAIN LOSS: 2.937988519668579
DEV LOSS: 3.285699222135544
----------------------------------------
..........E

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_two_cross, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50, sen_to_generate = 100)

0: I thought that this film was pretty boring. The only reason I watched it was because I was curious about it. But the acting wasn't too bad. The story is not that great either. The acting was good enough for a B-Movie movie. I
(tensor([0.3836, 0.1720, 0.0901, 0.0772, 0.0473], grad_fn=<ToCopyBackward0>), [' this', ' that', ' I', ' it', ' the'])
(tensor([0.3320, 0.2363, 0.0582, 0.0561, 0.0228], grad_fn=<ToCopyBackward0>), [' this', ' the', ' I', ' it', ' a'])
(tensor([0.4145, 0.1970, 0.1690, 0.0495, 0.0165], grad_fn=<ToCopyBackward0>), [' movie', ' film', ' was', ' is', ' would'])
(tensor([0.6391, 0.0585, 0.0580, 0.0559, 0.0176], grad_fn=<ToCopyBackward0>), [' was', ' is', ' had', ' would', ' could'])
(tensor([0.1007, 0.0720, 0.0701, 0.0441, 0.0339], grad_fn=<ToCopyBackward0>), [' a', ' very', ' pretty', ' so', ' really'])
(tensor([0.1270, 0.0969, 0.0869, 0.0650, 0.0502], grad_fn=<ToCopyBackward0>), [' bad', ' awful', ' funny', ' boring', ' atro'])
(tensor([0.3230, 0.2953, 0.1496, 0.04

In [ ]:
pytorch_stacked_triple_alt= Model_Import_6.MultiHeadModel_PyTorch_Stacked_Triple_Alt(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_triple_alt.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far

In [ ]:
train_one_style_w_dev(pytorch_stacked_triple_alt, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

.

KeyboardInterrupt: 

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_triple_alt, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 100)

0: I thought it would have a lot to say, and it has. It is not just about what it says about me. I am a person, a person with feelings and a life, a human. It's about the fact I have a family and I love my wife, my kids and I have friends and family and friends of my family and I love my friends. I have friends that are not like this, friends who have a lot more in their life. It is not about what it is.
0: I thought it would have a lot to say, and it has. It is not just about what it says about me. I am a person, a person with feelings and a life, a human. It's about the fact I have a family and I love my wife, my kids and I have friends and family and friends of my family and I love my friends. I have friends that are not like this, friends who have a lot more in their life. It is not about what it is.
0: I thought it would have a lot to say, and it has. It is not just about what it says about me. I am a person, a person with feelings and a life, a human. It's about the fact I have a

In [ ]:
pytorch_stacked_cross_positional= Model_Import_6.MultiHeadModel_PyTorch_Stacked_Positional(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_cross_positional.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far

In [ ]:
train_one_style_w_dev(pytorch_stacked_cross_positional, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.3605806827545166
DEV LOSS: 3.2910168653964997
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.248650312423706
DEV LOSS: 3.280663930606842
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.177448034286499
DEV LOSS: 3.2733029612064364
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 3.1034562587738037
DEV LOSS: 3.288507435274124
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 3.025240421295166
DEV LOSS: 3.30116226682663
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_cross_positional, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 100)

0: I thought it was good to have some of those guys in there, but I was really disappointed to not see a whole lot from them in this series, I mean I was hoping to have some good things from those two guys, and I was not. It's not like I was really disappointed with the rest, I was really looking for some help. I didn´ve seen any of those other two, and it´´´´d be good for me if I got a few more guys. But it didn't
WEIRD TOKEN PREDICTED
0: I thought it was good to have some of those guys in there, but I was a big part in getting the movie to where we are now, so it's not really fair. But it's not a good movie, but I don´ve got a feeling it will get a good review when the movie is released in theaters."I was in a relationship at that point and we had been dating a year, so it wasn't a good time. But it wasn´ve a lot of good times and a good movie
0: I thought it was good to have some of those guys in there, but I was really disappointed to not see a whole lot from them in this series, I

In [ ]:
pytorch_stacked_cross= Model_Import_6.MultiHeadModel_PyTorch_Stacked(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_cross.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


In [ ]:
train_one_style_w_dev(pytorch_stacked_cross, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.2936744689941406
DEV LOSS: 3.245574431180954
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.223997116088867
DEV LOSS: 3.2370047632217407
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.1751065254211426
DEV LOSS: 3.239879521083832
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 3.121413230895996
DEV LOSS: 3.2442128241062163
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 3.0633127689361572
DEV LOSS: 3.2566996613502504
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_cross, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

0: I now know what it's all like. It was like watching paint drying on the canvas. It's not the worst movie ever made. But I can only imagine the real story. I was very young, I didn`ve just graduated highschool and was working
0: I now know what it's all like. It was like watching paint drying on the wall of the house of the devil, the devil's own home, and the movie is so awful, I couldn`ve never even been a fan. The movie was awful,
0: I now know what it's all like. It was like watching paint drying on the canvas. It's not the worst movie ever made. But I can only imagine the real story. I was very young, I didn`ve just graduated highschool and was working
0: I now know what it's all like. It was like watching paint drying on the canvas. It's not the worst movie ever made. But I can only imagine the real story. I was very young, I didn`ve just graduated highschool and was working
0: I now know what it's all like. It was like watching paint drying on the canvas. It's not the worst mo

In [ ]:
pytorch_stacked_one_not_cross= Model_Import_6.MultiHeadModel_PyTorch_Stacked_One_Alt(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_one_not_cross.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


In [ ]:
train_one_style_w_dev(pytorch_stacked_one_not_cross, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 1.9614466428756714
DEV LOSS: 0.82565540587008
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 0.582233190536499
DEV LOSS: 0.5536075608596206
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 0.36249151825904846
DEV LOSS: 0.45461988470181824
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 0.27979013323783875
DEV LOSS: 0.4196639986075461
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 0.2359471619129181
DEV LOSS: 0.4072770089544356
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_one_not_cross, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 10, sen_to_generate = 1 )

0: The movie movie movie movie movie movie movie movie movie movie movie
(tensor([9.9798e-01, 1.0970e-03, 1.6808e-04, 1.4440e-04, 8.4563e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', ' movies', 'movie', ' the', ' theater'])
(tensor([9.9904e-01, 7.2250e-04, 4.6235e-05, 3.1694e-05, 1.5173e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', 'movie', ' movies', ' picture', ' theater'])
(tensor([9.9865e-01, 9.7820e-04, 4.1188e-05, 3.6795e-05, 2.6668e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', 'movie', ' movies', ' game', ' picture'])
(tensor([9.9883e-01, 7.1711e-04, 5.0114e-05, 4.7247e-05, 2.7799e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', 'movie', ' game', ' movies', ' picture'])
(tensor([9.9917e-01, 3.6677e-04, 4.7966e-05, 4.2018e-05, 2.5491e-05],
       device='cuda:0', grad_fn=<TopkBackward0>), [' movie', 'movie', ' game', ' movies', ' picture'])
(tensor([9.9896e-01, 3.2737e-04, 6.8437e-05, 5.9356e-05, 3.5

In [ ]:
pytorch_basic_positional= Model_Import_6.MultiHeadModel_PyTorch_Positional(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_basic_positional.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


In [ ]:
train_one_style_w_dev(pytorch_basic_positional, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 5, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.3547589778900146
DEV LOSS: 3.295817288970947
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.257972478866577
DEV LOSS: 3.276579099750519
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.1971757411956787
DEV LOSS: 3.2754206407546995
----------------------------------------
..........Epoch: 3, Epoch Examples: 10000
TRAIN LOSS: 3.1409406661987305
DEV LOSS: 3.2812213646888733
----------------------------------------
..........Epoch: 4, Epoch Examples: 10000
TRAIN LOSS: 3.0841259956359863
DEV LOSS: 3.29154849858284
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic_positional, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

NameError: name 'pytorch_basic_positional' is not defined

In [ ]:
num_heads = 8
pytorch_basic= Model_Import_6.MultiHeadModel_PyTorch(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_basic.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


In [ ]:
prompt = "I thought"

In [ ]:
train_one_style_w_dev(pytorch_basic, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 3, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

..........Epoch: 0, Epoch Examples: 10000
TRAIN LOSS: 3.2852327823638916
DEV LOSS: 3.245586234807968
----------------------------------------
..........Epoch: 1, Epoch Examples: 10000
TRAIN LOSS: 3.2262442111968994
DEV LOSS: 3.2330075203418733
----------------------------------------
..........Epoch: 2, Epoch Examples: 10000
TRAIN LOSS: 3.182225227355957
DEV LOSS: 3.2341347328186036
----------------------------------------


In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100,  num_tokens_to_generate = 50, sen_to_generate = 10 )

0: I thought this film was a good movie. I liked the cast, I liked the script. It's not the most entertaining movie ever made, but if you're looking for something to get you through a tough time in life, this one's for the job
(tensor([0.1569, 0.1447, 0.1324, 0.1013, 0.0751], grad_fn=<ToCopyBackward0>), [' I', ' this', ' it', ' that', ' the'])
(tensor([0.4233, 0.1911, 0.1710, 0.0260, 0.0237], grad_fn=<ToCopyBackward0>), [' movie', ' was', ' film', ' would', ' is'])
(tensor([0.5825, 0.0998, 0.0550, 0.0250, 0.0232], grad_fn=<ToCopyBackward0>), [' was', ' would', ' had', ' could', ' is'])
(tensor([0.1002, 0.0690, 0.0670, 0.0490, 0.0438], grad_fn=<ToCopyBackward0>), [' a', ' so', ' terrible', ' awful', ' bad'])
(tensor([0.0743, 0.0586, 0.0508, 0.0455, 0.0369], grad_fn=<ToCopyBackward0>), [' good', ' great', ' little', ' waste', ' disappointment'])
(tensor([0.1203, 0.1025, 0.0779, 0.0592, 0.0581], grad_fn=<ToCopyBackward0>), [' idea', ' example', ' one', ' movie', ' film'])
(tensor([0.2707,

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 39.41 GiB total capacity; 31.10 GiB already allocated; 3.56 MiB free; 31.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
pytorch_basic= Model_Import_6.MultiHeadModel_PyTorch(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
optimizer = optim.RAdam(pytorch_basic.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far


NameError: name 'num_heads' is not defined

In [ ]:
train_one_style_w_dev(pytorch_basic, optimizer, R_embeds_train, logits_train, token_ids_train, 5, logits_test, R_embeds_test,  token_ids_test) ## dev implemented

....................

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_mm)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, R_pos_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
MH_basic= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1]/num_heads)).to(device) #
# MH_basic= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8).to(device) #

neg_optimizer = optim.Adam(MH_basic.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style_w_dev(MH_basic, neg_optimizer, R_neg_embeds_train, neg_logits_train, neg_token_ids_train, 4, neg_logits_test, R_neg_embeds_test,  neg_token_ids_test) ## dev implemented

In [ ]:
####################### R_neg_embeds_train, R_neg_embeds_test, neg_logits_train, neg_logits_test, neg_token_ids_train, neg_token_ids_test
#(model, optimizer, context_embeds_list, logits_list, token_ids_list, epochs, dev_logits, dev_context, dev_token_ids, num_samples = 100)

In [ ]:
neg_alone_model = Model_Import_6.Test_skip_norm_model(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device)
neg_optimizer = optim.Adam(neg_alone_model.parameters(), lr=0.0001,  weight_decay=0.001)

In [ ]:
train_one_style(neg_alone_model, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
neg_alone_model_deeper = Model_Import_6.DeeperModel_skip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device)
neg_optimizer = optim.Adam(neg_alone_model_deeper.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(neg_alone_model_deeper, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
neg_alone_model_wide = Model_Import_6.WiderModel_skip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device) # 2 wide -> 1:46  4 wide ->3:00
neg_optimizer = optim.Adam(neg_alone_model_wide.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(neg_alone_model_wide, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
neg_alone_model_wide_deep = Model_Import_6.WiderDeeperModel_2(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device) #
neg_optimizer = optim.Adam(neg_alone_model_wide_deep.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(neg_alone_model_wide_deep, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
import torch
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

In [ ]:
WiderBlock_8_model= Model_Import_6.WiderBlock_8(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device) #
neg_optimizer = optim.Adam(WiderBlock_8_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(WiderBlock_8_model, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
one_head_8_wide_model= Model_Import_6.ProposedModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = neg_logits[0].shape[1]*8).to(device) #
neg_optimizer = optim.Adam(one_head_8_wide_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(one_head_8_wide_model, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
prompt = "The movie"

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, WiderBlock_8_model, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
prompt = "Overall I thought"

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, WiderBlock_8_model, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 100)

In [ ]:
prompt = "I sat"

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, one_head_8_wide_model, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
# WiderDeeperModel_Alt

In [ ]:
Wide_deep_alt= Model_Import_6.WiderDeeperModel_Alt(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = neg_logits[0].shape[1]).to(device) #
neg_optimizer = optim.Adam(Wide_deep_alt.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(Wide_deep_alt, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
narrow_M_head= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1]/4)).to(device) #
neg_optimizer = optim.Adam(narrow_M_head.parameters(), lr=0.00001,  weight_decay=0.001) # 2:00 per epoch

In [ ]:
train_one_style(narrow_M_head, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
 neg_logits[0].shape[1]/8

In [ ]:
R_neg_embeds[0].shape[0]/8

In [ ]:
Wide_S_head= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 1, attention_dim = int(neg_logits[0].shape[1]*8)).to(device) #
neg_optimizer = optim.Adam(Wide_S_head.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(Wide_S_head, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
wide_M_head= Model_Import_6.MultiHeadModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(wide_M_head.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(wide_M_head, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
wider_8_fixedM= Model_Import_6.WiderBlock_8_FixedM(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None).to(device) #
neg_optimizer = optim.Adam(wider_8_fixedM.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(wider_8_fixedM, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, wider_8_fixedM, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, narrow_M_head, Model_Import_6.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
middleskip_8= Model_Import_6.MultiHeadModel_WOFinalBlockSkip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(middleskip_8.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(middleskip_8, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
twoskip_8= Model_Import_6.MultiHeadModel_TwoSkip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(twoskip_8.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(twoskip_8, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
noskip= Model_Import_6.MultiHeadModel_WOAnyBlockSkip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(noskip.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(noskip, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
extraskip= Model_Import_6.MultiHeadModel_ExtraSkip(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = 8, attention_dim = int(neg_logits[0].shape[1])).to(device) #
neg_optimizer = optim.Adam(extraskip.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(extraskip, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)